In [2]:
import numpy as np

In [ ]:
# The parameters the paper said
print(1 + (1/2 + 4/5) * 1/50)
print((1.026)**50)

1.026
3.608843415268567


In [ ]:
# The parameters the paper actually used
print(1.34865**(1/50))
print(1.006**50)
print(0.006*50) # gamma + beta

1.0060000102257765
1.3486493145631329
0.3


In [ ]:
# precision scaling
print(100**(1/2))

10.0


In [7]:
x = 0
rho = (4-x**2)**(1/2) / (2 * np.pi)
print(rho)

np.random.normal(0, 1)

0.3183098861837907


0.9617375094946492

In [2]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter, FormatStrFormatter # For tick formatting

# =======================================================
# --- Functions ---
# =======================================================
def semicircle_pdf(x, radius=2):
    if np.abs(x) <= radius:
        return 2 * np.sqrt(radius**2 - x**2) / (radius**2 * np.pi)
    else:
        return 0

def sample_initial_conditions_semicircle_metropolis_hastings(N, radius=2, steps=1000, burn_in=100, proposal_std=0.5):
    X = np.zeros(N)
    x_current = 0  # Start at the center or a random point within [-radius, radius]

    # Burn-in phase
    for _ in range(burn_in):
        x_proposed = x_current + np.random.normal(0, proposal_std)
        if np.abs(x_proposed) > radius :
             pass # PDF will be 0, likely rejected if x_current is valid

        acceptance_ratio = 0
        pdf_current = semicircle_pdf(x_current, radius)
        if pdf_current > 0: # Avoid division by zero if x_current somehow became invalid
            pdf_proposed = semicircle_pdf(x_proposed, radius)
            acceptance_ratio = pdf_proposed / pdf_current
        
        if np.random.rand() < acceptance_ratio:
            x_current = x_proposed

    # Sampling phase
    for i in range(N):
        for _ in range(steps): # Run M-H for a few steps to get less correlated samples
            x_proposed = x_current + np.random.normal(0, proposal_std)

            acceptance_ratio = 0
            pdf_current = semicircle_pdf(x_current, radius)
            if pdf_current > 0:
                pdf_proposed = semicircle_pdf(x_proposed, radius)
                acceptance_ratio = pdf_proposed / pdf_current

            if np.random.rand() < acceptance_ratio:
                x_current = x_proposed
        X[i] = x_current
        
    # Check distribution
    plt.hist(X, bins=50, density=True, label='Sampled Initial Conditions')
    x_vals = np.linspace(-radius, radius, 200)
    y_vals = [semicircle_pdf(x, radius) for x in x_vals]
    plt.plot(x_vals, y_vals, label='True Semicircle PDF (scaled)')
    plt.legend()
    plt.title("Initial Condition Distribution Check")
    plt.show()
        
    return X

def solve_fully_coupled_system_euler(X0, beta, T, K, verbose=False):
    N = len(X0)
    X_ref = np.copy(X0)
    tau = T / K

    for k_step in range(K):
        X_new = np.zeros_like(X_ref) # Store updates for simultaneous update
        
        # Pre-calculate all interactions for efficiency if N is large,
        for i in range(N):
            interaction_force = 0
            for j in range(N):
                if i == j:
                    continue
                diff = X_ref[i] - X_ref[j]
                interaction_force += diff / (1.0 + diff**2)

            drift = -beta * X_ref[i] + (1.0 / (N - 1.0)) * interaction_force
            X_new[i] = X_ref[i] + drift * tau
        X_ref = X_new # Simultaneous update

        if verbose and k_step % (K // 10) == 0 and K >=10 :
             print(f"    Ref Sol: Step {k_step}/{K}")
    return X_ref


def solve_rbm1_euler(X0, beta, T, p, K, verbose=False):
    N = len(X0)
    X_rbm = np.copy(X0)
    tau = T / K

    for m in range(K):
        indices = np.random.permutation(N)
        
        X = np.copy(X_rbm)

        for i in range(0, N, p):
            batch_indices = indices[i : i + p]
            # Calculate drift for particles in this batch based on X_rbm (values at start of macro step)
            batch_drifts = np.zeros(p)
            X_batch = X_rbm[batch_indices]

            for i_idx, i in enumerate(batch_indices):
                current_drift = X_batch[i_idx]
                interaction_force = 0
                num_others = 0
                for j_idx, j in enumerate(batch_indices):
                    if i != j:
                        other_drift = X_batch[j_idx]
                        diff = current_drift - other_drift
                        interaction_force += diff / (1 + diff**2)
                        num_others += 1
                
                interaction_term = 0
                # Divisor is (p-1), i.e. num_others
                if num_others > 0:
                    interaction_term = (1 / num_others) * interaction_force
                
                batch_drifts[i_idx] = -beta * current_drift + interaction_term

            for i_idx, i in enumerate(batch_indices):
                X[i] = X_batch[i_idx] + batch_drifts[i_idx] * tau
        X_rbm = X

        if verbose and m % (K // 10) == 0 and K >= 10:
            print(f"    RBM Sol: Macro Step {m}/{K}")
            
    return X_rbm

def calculate_rbm_error(X_rbm, X_ref):
    if len(X_rbm) != len(X_ref):
        raise ValueError("RBM and Reference solutions must have the same number of particles.")
    if len(X_rbm) == 0:
        return 0
    error_sq_sum = np.sum((X_rbm - X_ref)**2)
    return np.sqrt(error_sq_sum / len(X_rbm))

In [3]:
import pickle

with open("my_session.pkl", "rb") as f:
    restored = pickle.load(f)

globals().update(restored)

print("✅ Variables restored into current workspace")


✅ Variables restored into current workspace
